---

## Set Up System Tools

[Homebrew](http://brew.sh/) installs the stuff you need that Apple didn’t. From a command line (denoted by `$`), enter the following:

```
$ /usr/bin/ruby -e "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/master/install)"
```

Insert the Homebrew directory at the top of your `PATH` environment variable by adding the following to your `~/.profile` (or `~/.zshrc`) file:

```
export PATH=/usr/local/bin:/usr/local/sbin:$PATH
```

Also install `git`, for version control and (later) access to the app's code:

```
$ brew install git
```

---

## Set Up PostgreSQL

Install with Homebrew:

```
$ brew update && brew install postgres
```

Or, if already installed,

```
$ brew update && brew upgrade postgres
```

Run `initdb` just once, basically to create the directory structure and such on disk that's needed for creating new databases. Note: The specified path should match the version of Postgres just installed!

```
$ initdb /usr/local/var/postgres9.5 -E utf8
```

To _manually_ start and stop a local Postgres server from running, use

```
$ pg_ctl -D /usr/local/var/postgres -l /usr/local/var/postgres/server.log start
$ pg_ctl -D /usr/local/var/postgres stop -s -m fast
```
        
Or to _automatically_ start a Postgres server (now and) at launch:

```
$ mkdir -p ~/Library/LaunchAgents
$ ln -sfv /usr/local/opt/postgresql/*.plist ~/Library/LaunchAgents
$ launchctl load ~/Library/LaunchAgents/homebrew.mxcl.postgresql.plist
```

Open the system paths file, `/etc/paths`, in a text editor, and move the line `/usr/local/bin` from the bottom of the file to the top (if it wasn't like this already). If you had to make a change to the file, reboot the computer. After rebooting, the command `which psql` command should return `/usr/local/bin/psql`.

Homebrew automatically created a database superuser account with the same login as your current Mac OS account. Let's create a dedicated user named `app` for connecting to and owning the app's database:

```
$ createuser --echo --pwprompt --superuser --createdb app
```

You'll be prompted to create a password — be sure to remember it or save it somewhere!

Homebrew also automatically created a database named `postgres` that may be used to log info for administrative tasks such as creating a user. Let's create a new database for this project:

```
$ createdb --echo --encoding=utf8 --host=<HOST> --port=<PORT> --username=app --owner=app <DBNAME>
```

We're using our `app` user to create the database, and assign it as the db's owner. **TODO:** Choose a `<DBNAME>` for this project!

To access the database through an interactive shell:

```
$ psql --host=<HOST> --port=<PORT> --username=app --dbname=<DBNAME>
```

While we're in the interactive shell, let's create the `pgcrypto` extension for secure password storage and `intarray` extension for integer array handling:

```
=# CREATE EXTENSION "pgcrypto";
=# CREATE EXTENSION "intarray";
```

Lastly, after exiting the postgres shell, define a `DATABASE_URL` environment variable containing this same information so that programs such as those in `cipy` can access it:

```
$ export DATABASE_URL=postgres://app:<PASSWORD>@<HOST>:<PORT>/<DBNAME>
```

---

## Set Up the App Code

First you'll need access permissions to the repo — contact DataKind! Then, "clone" the code needed to run the app from its repository hosted on GitLab. Make a new local directory for the repo (if needed) and change your current working directory to it:

```
$ mkdir /path/to/cloned_repo
$ cd /path/to/cloned_repo
$ git clone http://gitlab.datakind.org/conservation-intl/conservation-intl.git
```

This command should create a `conservation-intl` directory, in which the app's code lives.

---

## Set Up Python 3

For a fuller guide to everything below, check out [The Hitchhiker's Guide to Python](http://docs.python-guide.org/en/latest/).

Install Python 3 with Homebrew:

```
$ brew install python3
```

You may wish to work entirely within a virtual environment. (See [here](http://docs.python-guide.org/en/latest/dev/virtualenvs/?highlight=virtualenv) for more information on working within virtual envs.) If so, install with `pip`:

```
$ pip3 install virtualenv
```

You may wish to develop within an IPython interpreter and/or Jupyter Notebook:

```
$ pip3 install ipython jupyter
```

Change your working directory to your local copy of the repo (see the section above), then install all 3rd-party dependencies upon which the app will rely by:

```
$ cd /path/to/conservation-intl
$ pip install -r requirements.txt
```

Also add the repo to your `PYTHONPATH` environment variable by modifying the corresponding line (or adding a new line) in your `~/.profile` (or `~/.zshrc`) file:

```
export PYTHONPATH=/path/to/conservation-intl/:$PYTHONPATH
```

---

## Create Tables and Indexes from DDLs

** The code below is subject to change **

In [1]:
import logging

import cipy

In [2]:
logger = logging.getLogger('cipy')
logger.setLevel(logging.DEBUG)

In [3]:
conn_creds = cipy.db.get_conn_creds(env_var='DATABASE_URL')

for ddl_name in cipy.DEFAULT_DDL_PATHS:
    ddl = cipy.db.get_ddl(ddl_name)
    print('\n{}'.format(ddl))
    
    pgdb = cipy.db.PostgresDB(conn_creds, ddl=ddl)
    pgdb.create_table(act=True)
    print(ddl.create_table_statement())
    
    pgdb.create_indexes(act=True)
    for index_statement in ddl.create_index_statements():
        print(index_statement)


DDL(path='/Users/burtondewilde/Desktop/datakind/ci/conservation-intl/cipy/ddls/review_settings.yaml')
CREATE TABLE IF NOT EXISTS review_settings (review_id INTEGER PRIMARY KEY, num_citation_screening_reviewers INTEGER DEFAULT 2, num_fulltext_screening_reviewers INTEGER DEFAULT 2, required_citation_screener_id INTEGER DEFAULT NULL, required_fulltext_screener_id INTEGER DEFAULT NULL) 

DDL(path='/Users/burtondewilde/Desktop/datakind/ci/conservation-intl/cipy/ddls/citations.yaml')
CREATE TABLE IF NOT EXISTS citations (citation_id BIGSERIAL PRIMARY KEY, review_id INTEGER NOT NULL, user_id INTEGER NOT NULL, created_ts TIMESTAMP(0) WITHOUT TIME ZONE NOT NULL, type_of_work VARCHAR(25), title VARCHAR(250), secondary_title VARCHAR(250), publication_year SMALLINT, publication_month SMALLINT, authors VARCHAR(100) ARRAY, abstract TEXT, keywords VARCHAR(100) ARRAY, type_of_reference VARCHAR(50), journal_column_name VARCHAR(100), volume VARCHAR(20), issue_number VARCHAR(20), doi VARCHAR(100), issn 